In [1]:
import pandas as pd
import numpy as np

In [7]:
train = pd.read_csv('train_updated.csv')
train.head()

,day__0,day__1,day__2,day__3,day__4,day__5,day__6,day__7,hour__0,hour__1,...,title_count,genres_count,cities_count,tod_count,dow_count,segment,genres_sum,cities_sum,tod_time,dow_time
0,0,1,1,1,1,1,1,1,0,0,...,60,3,2,14,7,0,1456.416667,1456.433333,1456.283333,1456.266667
1,0,1,1,1,1,1,1,1,0,1,...,70,2,3,16,7,0,272.833333,272.850000,272.683333,272.733333
2,0,0,0,1,0,0,0,0,1,1,...,2,2,1,3,1,0,69.033333,69.033333,69.033333,0.050023
3,0,1,0,1,1,1,0,1,0,0,...,8,2,3,6,5,0,139.133333,139.133333,139.133333,139.133333
4,0,1,1,0,1,0,1,1,1,0,...,11,3,2,9,5,0,168.850000,168.866667,168.816667,168.816667


In [4]:
titles = pd.read_csv('new_titles.csv')
titles.head()

,t0,t1,t2,t3,t4,t5,t10,t12,t14,t17,...,t117,t131,t136,t141,t143,t146,t163,t167,t210,t253
0,0,0,0,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
city = pd.read_csv('added_cities.csv')
city = city.ix[:,'navi mumbai':]
city.head()

,navi mumbai,mumbai,delhi,bangalore,gurgaon,chennai,Unknown,kolkata,new delhi,pune,...,blacktown,bologna,boston,burwood east,cape town,clevedon,dadar,darjiling,dibrugarh,dimapur
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = pd.concat([train,titles],axis=1)
X.head()

,day__0,day__1,day__2,day__3,day__4,day__5,day__6,day__7,hour__0,hour__1,...,t117,t131,t136,t141,t143,t146,t163,t167,t210,t253
0,0,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [10]:
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import warnings; warnings.simplefilter('ignore')

In [11]:
features = list(X.columns)
features.remove('segment')

In [12]:
X_train = X[features]
y = X['segment']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X_train, y, test_size=0.20, random_state=42)

In [14]:
xgb = XGBClassifier()
xgb.fit(x_train,y_train)
pred = xgb.predict_proba(x_test)
score = roc_auc_score(y_test,pred[:,1])
print(score)

0.808091312887


In [16]:
col = list(city.columns)
max_score = score
max_score

0.80809131288671399

In [17]:
for i in col:
    x_train[i] = city.loc[:,i]
    x_test[i] = city.loc[:,i]
    xgb = XGBClassifier()
    xgb.fit(x_train,y_train)
    pred = xgb.predict_proba(x_test)
    score = roc_auc_score(y_test,pred[:,1])
    if(score>max_score):
        max_score = score
        print(i+" "+str(max_score))
    else:
        x_train = x_train.drop([i],axis=1)
        x_test = x_test.drop([i],axis=1)

navi mumbai 0.808111773071
mumbai 0.80840707078
bangalore 0.808890890151


KeyboardInterrupt: 

In [20]:
col.index('mysore')

182